In [1]:
from pinecone import Pinecone, ServerlessSpec
import os
from openai import OpenAI
import pandas as pd
from time import time
import dotenv
dotenv.load_dotenv()

True

In [2]:
token = os.getenv("RUNPOD_TOKEN")
open_ai_base_url = os.getenv("RUNPOD_EMBEDDING_URL")
model_name = os.getenv("MODEL_NAME")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
index_name = os.getenv("PINECONE_INDEX_NAME")

In [3]:
pc = Pinecone(api_key=pinecone_api_key)

client = OpenAI(
  api_key=token, 
  base_url=open_ai_base_url
)

# Try out embeddings

In [4]:
output = client.embeddings.create(input = ["helloo there"],model=model_name)
embedings = output.data[0].embedding
print(embedings)

[-0.05535165220499039, -0.056572869420051575, 0.08585154265165329, -0.06237364932894707, 0.01749393157660961, -0.011418377049267292, 0.052420731633901596, 0.052603915333747864, 0.02895810268819332, -0.02263830602169037, -0.013250201940536499, -0.04787169769406319, 0.029446590691804886, 0.031751636415719986, 0.05635915696620941, -0.009006474167108536, 0.012746450491249561, -0.054222024977207184, -0.103620246052742, -0.019554734230041504, 0.029156550765037537, 0.05199330672621727, -0.028805451467633247, -0.034804679453372955, 0.0015770869795233011, -0.006640366278588772, 0.019661590456962585, 0.03562900051474571, -0.008487456478178501, -0.07376149296760559, -0.00048514746595174074, -0.012944898568093777, 0.04658942297101021, -0.0003446598129812628, 0.051382698118686676, 0.0005123385926708579, 0.059625908732414246, -0.025065474212169647, -0.07791363447904587, -0.005609964486211538, 0.0604502335190773, -0.025019679218530655, -0.0016944382805377245, -0.016669608652591705, 0.0225619804114103

# Wrangle dataset

In [5]:
len(embedings)

384

In [6]:
df = pd.read_json("products/products.jsonl", lines=True)

In [7]:
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",375.0,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",275.0,4.3,SavoryScone.webp


In [8]:
df['text'] = df['name'] + ' : ' + df["description"]+\
    " -- Ingredients: " + df["ingredients"].astype(str)+\
    " -- Price: "+ df["price"].astype(str)+\
    " -- rating: " + df["rating"].astype(str)

In [9]:
df['text'].head(2)

0    Cappuccino : A rich and creamy cappuccino made...
1    Jumbo Savory Scone : Deliciously flaky and but...
Name: text, dtype: object

In [10]:
texts = df["text"].tolist()

In [11]:
with open("products/Misty_Brews_about_us.txt") as f:
    Misty_Brews_about_section = f.read()

Misty_Brews_about_section = "Coffee shop Misty Brews about section: " + Misty_Brews_about_section
texts.append(Misty_Brews_about_section)

In [12]:
texts

["Cappuccino : A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations. -- Ingredients: ['Espresso', 'Steamed Milk', 'Milk Foam'] -- Price: 375.0 -- rating: 4.7",
 "Jumbo Savory Scone : Deliciously flaky and buttery, this jumbo savory scone is filled with herbs and cheese, creating a mouthwatering experience. Perfect for a hearty snack or a light lunch, it pairs beautifully with your favorite coffee or tea. -- Ingredients: ['Flour', 'Butter', 'Cheese', 'Herbs', 'Baking Powder', 'Salt'] -- Price: 275.0 -- rating: 4.3",
 "Latte : Smooth and creamy, our latte combines rich espresso with velvety steamed milk, creating a perfect balance of flavor and texture. Enjoy it as a comforting treat any time of day, whether you're starting your morning or taking a midday break. -- Ingredients: ['Es

In [13]:
with open("products/menu_items_text.txt") as f:
    menu_items_text = f.read()

menu_items_text = "Menu: " + menu_items_text
texts.append(menu_items_text)

In [14]:
texts

["Cappuccino : A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations. -- Ingredients: ['Espresso', 'Steamed Milk', 'Milk Foam'] -- Price: 375.0 -- rating: 4.7",
 "Jumbo Savory Scone : Deliciously flaky and buttery, this jumbo savory scone is filled with herbs and cheese, creating a mouthwatering experience. Perfect for a hearty snack or a light lunch, it pairs beautifully with your favorite coffee or tea. -- Ingredients: ['Flour', 'Butter', 'Cheese', 'Herbs', 'Baking Powder', 'Salt'] -- Price: 275.0 -- rating: 4.3",
 "Latte : Smooth and creamy, our latte combines rich espresso with velvety steamed milk, creating a perfect balance of flavor and texture. Enjoy it as a comforting treat any time of day, whether you're starting your morning or taking a midday break. -- Ingredients: ['Es

# Generate Embeddings

In [15]:
output = client.embeddings.create(input = texts,model=model_name)

In [16]:
embeddings = output.data

In [17]:
embeddings[0]

Embedding(embedding=[-0.016249505802989006, -0.06927019357681274, 0.0036809328012168407, 0.005454646423459053, 0.02955426648259163, -0.043698202818632126, 0.038663145154714584, 0.014708473347127438, -0.02035384066402912, -0.04678026959300041, -0.0462004728615284, -0.031949732452631, -0.005054130684584379, -0.03848005086183548, 0.03176664188504219, -0.015433216467499733, -0.022688277065753937, -0.08617578446865082, -0.07189452648162842, -0.004100521095097065, 0.05849822238087654, -0.07262690365314484, -0.10601086169481277, -0.04061613604426384, 0.035489533096551895, -0.050075944513082504, 0.08111020922660828, -0.019865592941641808, -0.00033257133327424526, -0.13377997279167175, -0.009482692927122116, 0.00437897490337491, -0.023008691146969795, -0.030408700928092003, -0.03466561436653137, -0.0035531490575522184, 0.06829369813203812, -0.08239186555147171, 0.05596543475985527, -0.0006517920992337167, -0.007056710310280323, 0.03643551096320152, -0.06234317645430565, -0.002258147345855832, -

# Push data to database

In [18]:
pc.create_index(
    name = index_name,
    dimension=384,
    metric="cosine",
    spec= ServerlessSpec(
        cloud="aws",
        region="us-east-1",
    )
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-01', 'X-Cloud-Trace-Context': 'a4896b4389ad4e586edf0501cc807fea', 'Date': 'Sun, 16 Feb 2025 14:15:45 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [19]:
#wait for the index to be ready
while not pc.describe_index(index_name).status.ready:
    time.sleep(1)

index = pc.Index(index_name)

vectors = []
for text,e in zip(texts, embeddings):
    entry_id = text.split(":")[0]
    vectors.append(
        {
            "id": entry_id,
            "values": e.embedding,
            "metadata": {'text': text}
        }
    )

index.upsert(vectors=vectors, namespace='ns1')

{'upserted_count': 20}

# Get Closest Documents

In [20]:
output = client.embeddings.create(input = ["Is Cappuccino lactose-free?"],model=model_name)
embeding = output.data[0].embedding

In [21]:
results = index.query(
    namespace="ns1",
    vector=embeding,
    top_k=3,
    include_values=False,
    include_metadata=True
)

In [22]:
results

{'matches': [{'id': 'Cappuccino ',
              'metadata': {'text': 'Cappuccino : A rich and creamy cappuccino '
                                   'made with freshly brewed espresso, steamed '
                                   'milk, and a frothy milk cap. This '
                                   'delightful drink offers a perfect balance '
                                   'of bold coffee flavor and smooth milk, '
                                   'making it an ideal companion for relaxing '
                                   'mornings or lively conversations. -- '
                                   "Ingredients: ['Espresso', 'Steamed Milk', "
                                   "'Milk Foam'] -- Price: 375.0 -- rating: "
                                   '4.7'},
              'score': 0.732134283,
              'values': []},
             {'id': 'Menu',
              'metadata': {'text': 'Menu: Menu Items\n'
                                   '\n'
                              